In [33]:
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph
import collections
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#from cdlib import evaluation
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [34]:

'''
    Creates the benchmark network
    The seed parameter is not passed to create a different network all the time
    Sometimes if fails to create the network, so we give it a max of 30 times
'''
def create_benchmark_graph(n, tau1, tau2, mu, average_degree, min_community):
    count = 0
    num_tries = 30
    
    while count < num_tries:
        try:
            G = LFR_benchmark_graph(
                n, tau1, tau2, mu, average_degree=average_degree, \
                    min_community=min_community
            )
            return G
        except Exception as e: 
            print(e)
            count +=1 


In [35]:
'''
    All possible configuration we want to run our experiments
'''
configuration = {
    'n':[500,1581,5000,15811,50000], 
    'tau1': [3],
    'tau2': [1.5],
    'mu': [0.1],
    'avg_degree': [10]
}

In [36]:

configurations_possible = []

'''
    creates all the possible configuration objects taking the object
    configuration into account
'''
def get_possible_configs(conf, index, current):

    if index == len(conf):
        configurations_possible.append(current.copy())
        print(current)
        return 

    here = list(conf.keys())[index]

    for value in conf[here]:
        current[here] = value
        get_possible_configs(conf, index + 1, current)


get_possible_configs(configuration, 0, {})


{'n': 500, 'tau1': 3, 'tau2': 1.5, 'mu': 0.1, 'avg_degree': 10}
{'n': 1581, 'tau1': 3, 'tau2': 1.5, 'mu': 0.1, 'avg_degree': 10}
{'n': 5000, 'tau1': 3, 'tau2': 1.5, 'mu': 0.1, 'avg_degree': 10}
{'n': 15811, 'tau1': 3, 'tau2': 1.5, 'mu': 0.1, 'avg_degree': 10}
{'n': 50000, 'tau1': 3, 'tau2': 1.5, 'mu': 0.1, 'avg_degree': 10}


In [37]:
from collections import defaultdict
import community.community_louvain as community_louvain
import math

#from cdlib import algorithms
#import cdlib

'''
    Get the different dictionaries of the communities created by the LRF and 
    the ones gotten by runing the louvain algorithm
    Each dictionary key is going to correspond to a id of the community
'''
def get_communities(G):

    communities = {frozenset(G.nodes[v]["community"]) for v in G}


    # (com_nr, set community)
    lrf_dict = defaultdict()
    c = 1
    for y in communities:
        lrf_dict[c] = y
        c+= 1


    # (comm_nr, set communities)
    partion = community_louvain.best_partition(G)
    louvain_dict = defaultdict(set)


    for node, community in partion.items():
        louvain_dict[community].add(node) # (community_nr, nodes)

    #x1 = cdlib.NodeClustering(list(louvain_dict.values()), G)
    #x2 = cdlib.NodeClustering(list(lrf_dict.values()), G)
    #print(evaluation.normalized_mutual_information(x1, x2))


    return lrf_dict, louvain_dict


'''
    Having a set of community ids returns all the nodes of those communities
'''
def get_percentage_union(set_ids, mapping_ids):

    z = set()
    for id in set_ids:
        z = z.union(mapping_ids[id])
    return z


'''
    Checks if a community is considered small according to the value expected for the
    resolution limit to appear - sqt(2*m)
'''
def is_small_community(G, community):
    res_limit_val = math.sqrt(2*G.number_of_edges())
    sub = G.subgraph(community)
    return sub.number_of_edges() < res_limit_val


'''
    Returns the percentage between the identified communities that suffer from the resolution limit and
    the total number of identified communities
'''
def compare_communities_stats(G, threshold):

    lrf_dict, louvain_dict = get_communities(G)

    assigned_ones = defaultdict(set)# sets from the louvain that are assigned as most similar to the same set of the lrf

    for k, y in lrf_dict.items():
        
        higher = -1
        target_set = set() # set with the most common nodes
        overlapping_set_nr = set()

        for nr, com_set in louvain_dict.items():

            temp = com_set & y
            
            if len(temp) > 0:
                overlapping_set_nr.add(nr)
                assigned_ones[nr].add(k)

            
            if higher < len(temp):
                higher = len(temp)

        louvain_union = get_percentage_union(overlapping_set_nr, louvain_dict)
        lrf_union = lrf_dict[k]
        percentage = len(louvain_union.intersection(lrf_union)) / max(len(louvain_union), len(lrf_union))
        #print("For lrf community with size {}, there were found in louvain {} communities overlapping".format(len(y), len(overlapping_set_nr)))
        #print("For lrf community id {} (resol_lim : {}), they intersect with louvain communities ids {} => {}".format(k,\
        #is_small_community(G, lrf_union) ,overlapping_set_nr, percentage))

    counter = 0
    total = 0
    print("the other way around:")
    for k, y in assigned_ones.items(): 
        temp = []
        # k -> community id of louvain
        # y -> set of communities id of lrf
        # community id of louvain -> communties of lrf that overlap with the louvain
        louvain_union = louvain_dict[k]
        lrf_union = get_percentage_union(y, lrf_dict)
        percentage = len(louvain_union.intersection(lrf_union)) / max(len(louvain_union), len(lrf_union))
        #print("{} (louvain (resol_lim : {})) : {} (lrf) => {}" .format(k, is_small_community(G, louvain_union), y, percentage))

        for v in y:
            if len(lrf_dict[v] & louvain_union) / len(lrf_dict[v]) >= threshold:
                temp.append(v)
                total +=1
        
        if len(temp) > 1:
            #print(k, "=>", len(temp))
            counter += len(temp) #- 1
            #print(v, len(lrf_dict[v] & louvain_union) / len(lrf_dict[v]) )

    #print("counter total", counter, "|", len(lrf_dict), "|", counter/ len(lrf_dict))

    #print("counter2 total", counter, "|", total, "|", counter/total)

    if total == 0:
        return 0
        
    return counter/total

'''
    Compares the number of communities generated by the LRF-benchmark
    algorithm and the ones found by the louvain algorithm
'''
def compare_communities_number(G):
    partion = community_louvain.best_partition(G)
    l_c = max(partion.values())

    communities = {frozenset(G.nodes[v]["community"]) for v in G}
    lrf_c = len(communities)

    return l_c , lrf_c



In [38]:
'''
    Compares the community size distribution between the ones created by the louvain algorithm and 
    the lrf-benchmark algorithm. It also shows where the lower bound of the resolution limit stands
'''
def community_size_distribution(G):
    lrf_dict, louvain_dict = get_communities(G)
    counter=collections.Counter([G.subgraph(x).number_of_edges() for x in lrf_dict.values()])
    counter2=collections.Counter([G.subgraph(x).number_of_edges() for x in louvain_dict.values()])
    res_limit_val = math.sqrt(G.number_of_edges()/2)
    plt.figure(figsize=(6,6))
    plt.style.use('seaborn-whitegrid')
    plt.hist(counter,bins=20,alpha=0.6, label='LFR')
    plt.hist(counter2,bins=20,alpha=0.6, label='Louvain')
    plt.axvline(x=res_limit_val, color= 'black',linewidth=2,linestyle='--')
    plt.legend(loc='upper right')
    plt.show()

In [39]:
import networkx.algorithms.community as nxcomm

'''
    Calculates the percentage difference of the modularity between the louvain and the
    lrf-benchmark algorithm
'''
def modularity_dif(G):
    lrf_dict, louvain_dict = get_communities(G)
    mod_louvain = nxcomm.modularity(G,louvain_dict.values())
    mod_lfr = nxcomm.modularity(G,lrf_dict.values())
    #print(louvain_dict.values())
    #print(lrf_dict.values())
    #print(((mod_lfr-mod_louvain)*100)/abs((mod_louvain+mod_lfr)/2))
    return (((mod_lfr-mod_louvain)*100)/abs((mod_louvain+mod_lfr)/2))

In [40]:
'''
    Accepts a list of configuration and a callback function and runs the callback function
    for all those possible configurations
'''
def run_experiment(configurations_possible, experiment_call_back):

    values = []
    for configuration in configurations_possible:
        print(configuration)
        G = create_benchmark_graph(configuration['n'], configuration['tau1'], \
            configuration['tau2'], configuration['mu'], \
            average_degree=configuration['avg_degree'], min_community=40)
        values.append(experiment_call_back(G))
        #print(values)
    
    #print(values)
    return values

In [41]:
import numpy as np
from os.path import exists

'''
    runs the "run_experiment" function "times" times and stores those results in 
    a npy file
'''
def total_results(times,experiment_callback,path):
    if exists(path):
        results = np.load(path).tolist()
    else:
        results = []
    #print(len(results))
    i = 0
    while i < times:
        try:
            res = run_experiment(configurations_possible, experiment_callback)
            for j in range(len(res)):
                if len(results) <= j:
                    results.append([res[j]])
                else:
                    results[j].append(res[j])
            np.save(path,results)
            i+=1
        except Exception as e: 
            print(e)
    return results


In [42]:
import scipy.stats as st

'''
    By receiving a np array it calculate the t-distribution of that array with a 95% confidence level
'''
def get_intervals(res):
    intervals = []
    for data in res:
        if np.all(data == data[0]):
            intervals.append((data[0],data[0]))
        else:
            intervals.append(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))
    #intervals = [st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) for data in res]
    return intervals


In [43]:

'''
    Plots the values of our metric by showing the average and confidence intervals for each value
    It can also use the log scale and the chart can also be saved in the computer storage system
'''
def show_plot(x_values,intervals,mixing_param,metric= 'CCR', path='',x_log = False):
    y = [(x[0]+x[1])/2 for x in intervals]
    yerr = [(x[1]-x[0]) for x in intervals]
    plt.figure(figsize=(6,4))
    plt.errorbar(x_values, y, yerr=yerr, fmt='x--')
    plt.style.use('seaborn-whitegrid')
    if x_log:
        plt.xscale('log')
    plt.xlabel(mixing_param)
    plt.ylabel(metric)
    #plt.ylim(0)
    if path != '':
        plt.savefig(path+'.pgf')
    plt.show()


In [ ]:
# change the path accordingly
res = total_results(times = 20,experiment_callback = modularity_dif ,path='results/values/nodes_mod_test.npy')

In [ ]:
res_degree = np.load('average_degree.npy')
print(res_degree.shape)
intervals_degree = get_intervals(res_degree)
show_plot([6,10,14,18,22],intervals_degree,'<k>',path='results/pgf/mixing_k') #20000 nodes

In [ ]:
res_nodes = np.load('nodes.npy')
print(res_nodes.shape)
intervals_nodes = get_intervals(res_nodes)
show_plot([500,1581,5000,15811,50000],intervals_nodes,'N',x_log=True,path='results/pgf/mixing_N') #avg_degree 10

In [ ]:
res_degree = np.load('mu.npy')
print(res_degree.shape)
intervals_degree = get_intervals(res_degree)
show_plot([0.1,0.2,0.3,0.4,0.5],intervals_degree,'$\mu$',path='results/pgf/mixing_mu') #avg_degree 10

In [ ]:
res_degree = np.load('tau1.npy')
print(res_degree.shape)
intervals_degree = get_intervals(res_degree)
show_plot([2.6,2.7,2.8,2.9,3],intervals_degree,'$\gamma$',path='results/pgf/mixing_tau1') #avg_degree 8

In [ ]:
res_degree = np.load('tau2.npy')
print(res_degree.shape)
intervals_degree = get_intervals(res_degree)
show_plot([1.2,1.4,1.6,1.8,2],intervals_degree,'$B$',path='results/pgf/mixing_tau2')  #avg_degree 8

In [ ]:
res_degree = np.load('nodes_mod.npy')
print(res_degree)
intervals_degree = get_intervals(res_degree)
show_plot([500,1581,5000,15811,50000],intervals_degree,'N',x_log=True,metric='Modularity Diference (%)',path='results/pgf/gmixing_N_mod')

In [ ]:
res_degree = np.load('average_degree_mod.npy')
print(res_degree.shape)
intervals_degree = get_intervals(res_degree)
show_plot([6,10,14,18,22],intervals_degree,'<k>',metric='Modularity Diference (%)')

In [ ]:
counter_lfr=[]
counter_louvain=[]
res_limit_val = []

'''
    Obtain the values for the distribution of the number of internal links per community graph for the louvain and
    lrf-benchmark algorithm
'''
def mu_distribution():
    counter_lfr=[]
    counter_louvain=[]
    res_limit_val = []
    for configuration in configurations_possible:
        print(configuration)
        G = create_benchmark_graph(configuration['n'], configuration['tau1'], \
            configuration['tau2'], configuration['mu'], \
            average_degree=configuration['avg_degree'], min_community=40)
        lrf_dict, louvain_dict = get_communities(G)
        counter_lfr.append(collections.Counter([G.subgraph(x).number_of_edges() for x in lrf_dict.values()]))
        counter_louvain.append(collections.Counter([G.subgraph(x).number_of_edges() for x in louvain_dict.values()]))
        res_limit_val.append(math.sqrt(G.number_of_edges()/2))
    return counter_lfr,counter_louvain,res_limit_val
counter_lfr,counter_louvain,res_limit_val = mu_distribution() 

In [50]:
'''
    Plot the graph  for the distribution of the number of internal links per community graph for the louvain and
    lrf-benchmark algorithm
'''
def mu_distribution_plot():
    fig, axs = plt.subplots(2, 2,figsize=(9,7))

    bins=np.histogram(np.hstack((list(counter_lfr[0].keys()),list(counter_louvain[0].keys()))), bins=30)[1]
    axs[0,0].hist(counter_lfr[0],bins=bins,alpha=0.6, label='LFR')
    axs[0,0].hist(counter_louvain[0],bins=bins,alpha=0.6, label='Louvain')
    axs[0,0].axvline(x=res_limit_val[0], color= 'black',linewidth=1,linestyle='--')
    axs[0,0].text(0.6, 0.6, '$\mu = 0.2$', fontsize=13, transform=axs[0,0].transAxes)
    axs[0,0].set_yscale('log')
    axs[0,0].legend(loc='upper right')

    bins=np.histogram(np.hstack((list(counter_lfr[1].keys()),list(counter_louvain[1].keys()))), bins=30)[1]
    axs[0,1].hist(counter_lfr[1],bins=bins,alpha=0.6, label='LFR')
    axs[0,1].hist(counter_louvain[1],bins=bins,alpha=0.6, label='Louvain')
    axs[0,1].axvline(x=res_limit_val[1], color= 'black',linewidth=1,linestyle='--')
    axs[0,1].text(0.6, 0.6, '$\mu = 0.3$', fontsize=13, transform=axs[0,1].transAxes)
    axs[0,1].set_yscale('log')
    axs[0,1].legend(loc='upper right')

    bins=np.histogram(np.hstack((list(counter_lfr[2].keys()),list(counter_louvain[2].keys()))), bins=30)[1]
    axs[1,0].hist(counter_lfr[2],bins=bins,alpha=0.6, label='LFR')
    axs[1,0].hist(counter_louvain[2],bins=bins,alpha=0.6, label='Louvain')
    axs[1,0].axvline(x=res_limit_val[2], color= 'black',linewidth=1,linestyle='--')
    axs[1,0].text(0.6, 0.6, '$\mu = 0.4$', fontsize=13, transform=axs[1,0].transAxes)
    axs[1,0].set_yscale('log')
    axs[1,0].legend(loc='upper right')

    bins=np.histogram(np.hstack((list(counter_lfr[3].keys()),list(counter_louvain[3].keys()))), bins=30)[1]
    axs[1,1].hist(counter_lfr[3],bins=bins,alpha=0.6, label='LFR')
    axs[1,1].hist(counter_louvain[3],bins=bins,alpha=0.6, label='Louvain')
    axs[1,1].axvline(x=res_limit_val[3], color= 'black',linewidth=1,linestyle='--')
    axs[1,1].text(0.6, 0.6, '$\mu = 0.5$', fontsize=13, transform=axs[1,1].transAxes)
    axs[1,1].set_yscale('log')
    axs[1,1].legend(loc='upper right')

    for ax in axs.flat:
        ax.set(xlabel='Internal links', ylabel='frequency')
    plt.show()

In [ ]:

mu_distribution_plot()